In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [6]:
import gradio as gr 

In [7]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [8]:
openai = OpenAI()

In [9]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
message_gpt("What is today's date?")

"Today's date is June 13, 2024."

In [11]:
def shout(text):
    print(f"Shout has beeb called with input: {text}")
    return text.upper()

In [12]:
shout("hello")

Shout has beeb called with input: hello


'HELLO'

In [13]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has beeb called with input: dsdsd
